In [3]:
from cortical_tools.datasets.v1dd import client
# from cortexclient.mesh_vertex import *

# rid = 864691132544823185
# vass = VertexAssigner(root_id=rid, caveclient=client.cave)

In [4]:
rid = 864691132544823185

mapping, vass = client.mesh.compute_vertex_to_l2_mapping(
    rid,
    return_assigner=True
)

Processing complex chunks...:   0%|          | 0/744 [00:00<?, ?it/s]

In [5]:
pf_df = client.tables.proofreading_status_and_strategy(status_axon=True).query()

In [7]:
client.mesh.mesh_l2_mappings[rid]

array([157013146664435759, 157013146664435759, 157013146664435759, ...,
       171579752191100135, 171579752191100135, 171579752191100135],
      shape=(1126948,))

In [7]:
import pandas as pd
import numpy as np
from nglui import skeletons


In [9]:
chunk_df = vass.chunk_df.copy()
chunk_df['b'] = np.random.random(chunk_df.shape[0])
chunk_df['g'] = np.random.random(chunk_df.shape[0])
chunk_df['r'] = np.random.random(chunk_df.shape[0])

vert_df = pd.DataFrame(
    {
        'point_x': vass.vertices[:,0],
        'point_y': vass.vertices[:,1],
        'point_z': vass.vertices[:,2],
        'mesh_label': vass.mesh_label_index,
    }
)

vert_df = vert_df.merge(
    chunk_df[['r', 'g', 'b']],
    right_index=True,
    left_on='mesh_label',
    how='left',
)

vert_df = vert_df.fillna(1)

skltor = skeletons.SkeletonManager(
    client.cave,
    cloudpath='gs://csm-skel/ngl3',
    vertex_attributes=['r', 'g', 'b',],
    initialize_info=True,
)
skltor.upload_skeleton(
    root_id = rid,
    vertices= vass.vertices,
    edges = [],
    vertex_attribute_data=vert_df[['r', 'g', 'b']],
)

In [10]:
rid

864691132544823185

In [ ]:
grps = [grp for _, grp in vass.chunk_df_multi.groupby('chunk_number')]

In [ ]:
import tqdm

In [ ]:
for grp in tqdm.tqdm(grps):
    vass.process_multicomponent_chunk(
        grp,
        vass.vertices,
        vass.faces,
        ts = vass.timestamp,
    )

In [ ]:
grp

In [ ]:
vass.process_multicomponent_chunk(
    grp,
    vass.vertices,
    vass.faces,
    ts = vass.timestamp,
)

In [ ]:
chunk_rows = grp
vertices = vass.vertices
faces = vass.faces

In [ ]:
pts = np.array(chunk_rows[["pt_x", "pt_y", "pt_z"]].values, dtype=float)
components = create_component_dict(chunk_rows, vertices, faces)


In [ ]:
components

In [ ]:
mask_all = bbox_mask(chunk_rows.iloc[0], vertices, inclusive=True)
vertices_chunk = vertices[mask_all]
faces_filter = faces[np.all(mask_all[faces], axis=1)]
relabel = {v: k for k, v in enumerate(np.flatnonzero(mask_all))}
faces_chunk = fastremap.remap(
    faces_filter,
    relabel,
)


In [ ]:
mask_in = bbox_mask(chunk_rows.iloc[0], vertices_chunk, inclusive=False)

In [ ]:
faces_not_touching_edge = faces_chunk[np.all(mask_in[faces_chunk], axis=1)]

In [ ]:
face_identity = np.array([[ii, ii, ii] for ii in range(mask_in.shape[0])])


In [ ]:
vertex_cc = gyp.connected_components(
    np.vstack(
        [np.atleast_2d(faces_not_touching_edge).reshape(-1, 3), face_identity]
    )
)


In [ ]:
vertex_cc[~mask_in] = -1


In [ ]:
components = []
assigned_vertices = np.full(mask_all.shape[0], False, dtype=bool)
comp_id = 0


In [ ]:
for ii in np.unique(vertex_cc[mask_in]):
    comp_mask = np.full(mask_all.shape, False, dtype=bool)
    comp_mask[mask_all] = vertex_cc == ii
    comp_verts, comp_faces = component_submesh(
        vertex_cc == ii, vertices_chunk, faces_chunk
    )
    assigned_vertices[comp_mask] = True
    if comp_faces.shape[0] == 0:
        continue
    components.append(
        {
            "component_id": comp_id,
            "vertices": comp_verts,
            "faces": comp_faces,
            "mask": comp_mask,
            "vertices_in": vertices[comp_mask],
        }
    )
    comp_id += 1


In [ ]:
components

In [ ]:
faces_chunk_mask = np.flatnonzero(np.any(mask_in[vass.faces], axis=1))

In [ ]:
minds = np.flatnonzero(mask_in)
relabel = {v: k for k, v in enumerate(minds)}


In [ ]:
face_mask = mask_in[vass.faces]
faces_relabel = fastremap.remap(
    vass.faces[np.all(face_mask, axis=1)],
    relabel,
)

In [ ]:
face_identity = np.array([[ii, ii, ii] for ii in range(len(minds))])
vertex_cc = gyp.connected_components(
    np.vstack([np.atleast_2d(faces_relabel).reshape(-1, 3), face_identity])
)


In [ ]:
components = []
assigned_vertices = np.full(mask_in.shape[0], False, dtype=bool)
comp_id = 0


In [ ]:
ii = 0

In [ ]:
comp_verts, comp_faces, comp_mask = component_submesh(
            vertex_cc == ii, mask_in, vass.vertices, vass.faces, faces_chunk_mask
        )

In [ ]:
component_mask = mask_in.copy()
component_mask[mask_in] = vertex_cc == ii

In [ ]:
face_touch_component = np.full(vass.faces.shape[0], False, dtype=bool)

In [ ]:
chunk_faces = vass.faces[faces_chunk_mask]

In [ ]:
within_component_mask = vertex_cc == ii

In [ ]:
relabel = np.where()

In [ ]:
within_component_mask.shape

In [ ]:
remap_dict = {k: ii for ii, k in enumerate(np.flatnonzero(within_component_mask))}
chunk_faces_short = fastremap.remap(
    chunk_faces, remap_dict
)

In [ ]:
i

In [ ]:
%%timeit
np.flatnonzero(mask_in[vass.faces].sum(axis=1)>0)

In [ ]:
%%timeit
faces_touching_vertices_numba(
    vass.faces,
    np.flatnonzero(mask_in),
)

In [ ]:
%%timeit
np.any(mask_in[vass.faces], axis=1)

In [ ]:
from nglui import statebuilder

In [ ]:
(mesh_label == -1).sum()

In [ ]:
mesh_label = vass.lvl2_map()
chunk_df = vass.chunk_df.copy()
chunk_df['b'] = np.random.random(chunk_df.shape[0])
chunk_df['g'] = np.random.random(chunk_df.shape[0])
chunk_df['r'] = np.random.random(chunk_df.shape[0])

vert_df = pd.DataFrame(
    {
        'point_x': vass.vertices[:,0],
        'point_y': vass.vertices[:,1],
        'point_z': vass.vertices[:,2],
        'mesh_label': vass.mesh_label,
    }
)

vert_df = vert_df.merge(
    chunk_df[['r', 'g', 'b']],
    right_index=True,
    left_on='mesh_label',
    how='left',
)

vert_df = vert_df.fillna(1)
from nglui import skeletons
skltor = skeletons.SkeletonManager(
    client.cave,
    cloudpath='gs://csm-skel/ngl3',
    vertex_attributes=['r', 'g', 'b',],
    initialize_info=True,
)
skltor.upload_skeleton(
    root_id = rid,
    vertices= vass.vertices,
    edges = [],
    vertex_attribute_data=vert_df[['r', 'g', 'b']],
)

In [ ]:
skltor.cloudpath

In [ ]:
rid

In [ ]:
vass.vertices.shape

In [ ]:
np.unique(vass.mesh_label)

In [ ]:
client.neuroglancer_url(clipboard=True)

In [ ]:
(vass.mesh_label == -1).sum()

In [ ]:
for idx, l2id in vass.chunk_df['l2id'].items():
    skltor.upload_skeleton(
        root_id = l2id,
        vertices= vass.vertices[mesh_label == idx],
        edges = [],
        vertex_attribute_data=vert_df.query("mesh_label == @idx")[['r', 'g', 'b']],
    )

In [ ]:
rid

In [ ]:
vass.chunk_df.query('l2id==168346295256221013')

In [ ]:
vass.chunk_df_multi.query('chunk_number == 361071634')

In [ ]:
grps = [grp for _, grp in vass.chunk_df_multi.groupby('chunk_number')]

In [ ]:
grps[43]

In [ ]:
cv = client.cave.info.segmentation_cloudvolume()

In [ ]:
ts = client.now() - datetime.timedelta(days=4*365)

In [ ]:
client.cave.chunkedgraph.get_root_timestamps(169402444692849255)[0] - datetime.timedelta(seconds=1)

In [ ]:
cv.scattered_points([101197, 107655, 7874], agglomerate=True, stop_layer=2, timestamp=client.cave.chunkedgraph.get_root_timestamps(169402444692849255)[0] - datetime.timedelta(seconds=1))

In [ ]:
cv.scattered_points([101197, 107655, 7874], agglomerate=True, stop_layer=2,)

In [ ]:
client.cave.chunkedgraph.get_roots()

In [ ]:
chunk_rows = vass.chunk_df_multi.query('chunk_number == 361071634')
pts = chunk_rows[['pt_x', 'pt_y', 'pt_z']].values
vertices = vass.vertices
faces = vass.faces
ts = vass.timestamp

In [ ]:
components, unassigned_mask = create_component_dict(chunk_rows, vertices, faces)

wn_results = []
for comp in components:
    wn_results.append(
        gyp.fast_winding_number(pts, comp["vertices"], comp["faces"])
    )
wn_results = np.array(wn_results).T

pt_assign, mesh_assign = linear_sum_assignment(
    np.array(wn_results) / np.max(wn_results), maximize=True
)
if len(pts) < len(components):
    mesh_assign = mesh_assign[: len(pt_assign)]
if len(pts) > len(components):
    pt_assign = pt_assign[: len(mesh_assign)]
result_df = pd.DataFrame(
    {
        "representative_pt": pt_assign.astype(int),
        "graph_comp": mesh_assign.astype(int),
    }
)
result_df

In [ ]:
from scipy import spatial

In [ ]:
max_distance = 400
ratio_better = 0.25

assigned_components = result_df["graph_comp"].unique()

first_comps = []
second_comps = []
first_assigned = []
second_assigned = []
ds = []
kdtrees = [spatial.KDTree(comp["vertices"]) for comp in components]
for comp_a, comp_b in combinations(components, 2):
    first_comps.append(comp_a["component_id"])
    second_comps.append(comp_b["component_id"])
    first_assigned.append(comp_a["component_id"] in assigned_components)
    second_assigned.append(comp_b["component_id"] in assigned_components)
    if not (
        comp_a["component_id"] in assigned_components
        and comp_b["component_id"] in assigned_components
    ):
        comp_ds = np.array(list(kdtrees[comp_a['component_id']].sparse_distance_matrix(
            kdtrees[comp_b['component_id']], max_distance=max_distance / ratio_better, output_type='dok_matrix'
        ).values()))
        if len(comp_ds) > 0:
            ds.append(np.min(comp_ds))
        else:
            ds.append(np.inf)
    else:
        ds.append(np.inf)
distance_graph = pd.DataFrame(
    {
        "first_comp": first_comps,
        "second_comp": second_comps,
        "first_assigned": first_assigned,
        "second_assigned": second_assigned,
        "distance": ds,
    }
)
distance_graph = distance_graph[distance_graph["distance"] < max_distance]
distance_graph["evaluated"] = False

In [ ]:
result_df

In [ ]:
while not np.all(distance_graph["evaluated"]):
    pairs_to_consider = distance_graph.query(
        "evaluated == False and first_assigned != second_assigned"
    ).sort_values("distance")
    if len(pairs_to_consider) == 0:
        break
    for gph_idx, row in pairs_to_consider.iterrows():
        distance_graph.loc[gph_idx, "evaluated"] = True
        if row["first_assigned"]:
            assigned_comp = row["first_comp"]
            unassigned_comp = row["second_comp"]
        else:
            assigned_comp = row["second_comp"]
            unassigned_comp = row["first_comp"]
        ds_edge = (
            pairs_to_consider.drop(index=gph_idx)
            .query(
                "first_comp == @unassigned_comp or second_comp == @unassigned_comp and evaluated == False and first_assigned!=second_assigned"
            )["distance"]
            .values
        )
        do_assign = False
        if len(ds_edge) == 0:
            do_assign = True
        elif row["distance"] < ratio_better * np.min(ds_edge):
            do_assign = True
        if do_assign:
            best_pt = result_df.query("graph_comp == @assigned_comp")[
                "representative_pt"
            ].values[0]
            result_df.loc[result_df.index[-1] + 1] = {
                "representative_pt": best_pt,
                "graph_comp": unassigned_comp,
            }
            distance_graph.loc[
                distance_graph["first_comp"] == unassigned_comp,
                "first_assigned",
            ] = True
            distance_graph.loc[
                distance_graph["second_comp"] == unassigned_comp,
                "second_assigned",
            ] = True


In [ ]:
result_df

In [ ]:
distance_graph

In [ ]:
len(components) > len(pt_assign)

In [ ]:
len(components)

In [ ]:
max_distance = 400
ratio_better = 0.25

In [ ]:
result_df

In [ ]:
vass.representative_point_via_proximity(
    components=components,
    result_df=result_df,
    max_distance=max_distance,
    ratio_better=ratio_better,
)aa
result_df

In [ ]:
assigned_components = result_df["graph_comp"].unique()
if len(assigned_components) == 0:
    print('this one')
vert_assigned = {
    comp["component_id"]: comp["vertices_in"]
    for comp in components
    if comp["component_id"] in assigned_components
}


In [ ]:
unassigned_components = [
    comp
    for comp in components
    if comp["component_id"] not in assigned_components
]


In [ ]:
vass.find_closest_assigned_component(
    unassigned_components[0],
    vert_assigned,
    max_distance=400,
    ratio_better=0.25,
)

In [ ]:
comp = unassigned_components[0]


In [ ]:
if len(vert_assigned) == 1:
    print('nuttin')
ds = np.array(
    [np.min(cdist(comp["vertices"], v)) for v in vert_assigned.values()]
)
print(ds)
dist_sort = np.argsort(ds)
if (
    ds[dist_sort[0]] < ratio_better * ds[dist_sort[1]]
    and ds[dist_sort[0]] < max_distance
):
    print(list(vert_assigned.keys())[dist_sort[0]])

In [ ]:
ds = np.array([np.min(cdist(comp["vertices"], v)) for v in vert_assigned.values()])
# ds[ds == 0] = np.inf  # Ignore zero distances, since they would be attached
dist_sort = np.argsort(ds)

In [ ]:
ds

In [ ]:
pt_assigned = vass.representative_point_via_proximity(
    components=components,
    result_df=result_df,
    max_distance=250,
    ratio_better=0.25,
)


In [ ]:
pt_assigned

In [ ]:
skltor.cloudpath

In [ ]:
vs = (
    statebuilder.ViewerState(client=client.cave)
    .add_layers_from_client(segmentation='seg')
    .add_segments([rid])
    .add_points(
        point_column='pt',
        data_resolution=[1,1,1],
        segment_column='l2id',
        data=vass.chunk_df,
    )
).to_clipboard(shorten=True)


In [ ]:
cv = client.cave.info.segmentation_cloudvolume()

In [ ]:
cv.parallel = 10

In [ ]:
meshes = client.mesh.get_meshes(l2ids)

In [ ]:
chunk_df.query('l2id == 166086454928212643')

In [ ]:
components, unassigned_mask = create_component_dict(
    chunk_rows,
    vass._vertices,
    vass._faces,
)

In [ ]:
components[0].keys()

In [ ]:
len(components)

In [ ]:
pts = np.array(chunk_rows[["pt_x", "pt_y", "pt_z"]].values, dtype=float)

wn_results = []
for comp in components:
    wn_results.append(
        gyp.fast_winding_number(pts, comp["vertices"], comp["faces"])
    )
wn_results = np.array(wn_results).T


In [ ]:
pt_assign, mesh_assign = linear_sum_assignment(
    np.array(wn_results) / np.max(wn_results), maximize=True
)

if len(pts) < len(components):
    mesh_assign = mesh_assign[: len(pt_assign)]
if len(pts) > len(components):
    pt_assign = pt_assign[: len(mesh_assign)]
result_df = pd.DataFrame(
    {
        "representative_pt": pt_assign.astype(int),
        "graph_comp": mesh_assign.astype(int),
    }
)


In [ ]:
result_df

In [ ]:
if len(components) > len(pt_assign):
    for comp in components:
        # If you have not already assigned a point to this component, use the slower cloudvolume lookup
        if comp["component_id"] not in result_df["graph_comp"].values:
            point_to_component = vass.representative_point_via_lookup(
                chunk_rows=chunk_rows,
                comp=comp,
                timestamp=vass._timestamp,
            )
            if point_to_component == -1:
                continue
            result_df.loc[result_df.index[-1] + 1] = {
                "representative_pt": point_to_component,
                "graph_comp": comp["component_id"],
            }


In [ ]:
chunk_rows

In [ ]:
l2ids = chunk_rows["l2id"].values
l2id = vass.get_mesh_l2id_from_lookup(
    comp,
    point_counts=[10, 50, 100, 200, 500],
    potential_l2ids=l2ids,
    timestamp=vass._timestamp,
)


In [ ]:
vass.representative_point_via_lookup(
    chunk_rows=chunk_rows,
    comp=components[0],
    timestamp=vass._timestamp,
)

In [ ]:
l2ids = chunk_rows["l2id"].values

In [ ]:
l2id = vass.get_mesh_l2id_from_lookup(
    comp,
    point_counts=[10, 50, 100, 200, 500],
    potential_l2ids=l2ids,
    timestamp=vass._timestamp,
)


In [ ]:
l2id

In [ ]:
point_to_component

In [ ]:
[c['component_id'] for c in components]

In [ ]:
wn_results

In [ ]:
vass.process_multicomponent_chunk(
    chunk_rows,
    vass._vertices,
    vass._faces,
    vass._timestamp,
)

In [ ]:
id_mapping = []
for _, chunk_rows in tqdm(vass._chunk_df_multi.groupby("chunk_number")):
    id_mapping.extend(
        vass.process_multicomponent_chunk(
            chunk_rows, vass._vertices, vass._faces, vass._timestamp
        )
    )

In [ ]:
grps = [grp for _, grp in vass._chunk_df_multi.groupby("chunk_number")]

In [ ]:
grp = grps[484]
vass.process_multicomponent_chunk(
    grp, vass._vertices, vass._faces, vass._timestamp
)

In [ ]:
assignment_df, component_mask_dict, unassigned_mask = (
    vass.assign_points_to_components(
        grp,
        vass._vertices,
        vass._faces,
        vass._timestamp,
    )
)

chunk_rows = grp
pts = np.array(chunk_rows[["pt_x", "pt_y", "pt_z"]].values, dtype=float)

In [ ]:
components, unassigned_mask = create_component_dict(chunk_rows, vass._vertices, vass._faces)

In [ ]:
len(components)

In [ ]:
chunk_rows

In [ ]:
[len(c['vertices']) for c in components]

In [ ]:
wn_results = []
for comp in components:
    wn_results.append(
        gyp.fast_winding_number(pts, comp["vertices"], comp["faces"])
    )
wn_results = np.array(wn_results).T
pt_assign, mesh_assign = linear_sum_assignment(
    np.array(wn_results) / np.max(wn_results), maximize=True
)
if len(pts) < len(components):
    mesh_assign = mesh_assign[: len(pt_assign)]
if len(pts) > len(components):
    print(
        f"More points than components for chunk number {chunk_rows.iloc[0, 'chunk_number']}"
    )
    pt_assign = pt_assign[: len(mesh_assign)]
result_df = pd.DataFrame(
    {
        "representative_pt": pt_assign.astype(int),
        "graph_comp": mesh_assign.astype(int),
    }
)

In [ ]:
wn_results

In [ ]:
result_df

In [ ]:
pt_assign, mesh_assign

In [ ]:
component_mask_dict

In [ ]:
unassigned_mask.any()

In [ ]:
df_solo, df_multi = vass.get_chunk_dataframes(rid)

In [ ]:
id_mapping_solo = vass.process_solo_chunks(df_solo)

In [ ]:
l2ids, l2_pts = vass.get_l2_components(rid)
vertices, faces = vass.get_mesh_data(rid)

In [ ]:
chunk_df_solo

In [ ]:
chunk_df = vass.chunk_dataframe(l2ids, l2_pts)

chunk_df_solo = chunk_df.drop_duplicates('chunk_number', keep=False)
mesh_label = np.zeros(mesh.vertices.shape[0], dtype=int)-1
for i, row in tqdm(chunk_df_solo.iterrows()):
    mask = bbox_mask(row, mesh.vertices)
    mesh_label[mask] = i

chunk_df_multi = chunk_df[chunk_df.duplicated('chunk_number', keep=False)]
grps = [group for _, group in chunk_df_multi.groupby('chunk_number')]

In [ ]:
client.cave.chunkedgraph.get_leaves(rid, stop_layer=2)

In [ ]:
l2ids = client.cave.chunkedgraph.get_
pts = get_lvl2_points(l2ids, client.cave)

chunk_df = chunk_dataframe(l2ids, pts, cv)

chunk_df_solo = chunk_df.drop_duplicates('chunk_number', keep=False)
mesh_label = np.zeros(mesh.vertices.shape[0], dtype=int)-1
for i, row in tqdm(chunk_df_solo.iterrows()):
    mask = bbox_mask(row, mesh.vertices)
    mesh_label[mask] = i

chunk_df_multi = chunk_df[chunk_df.duplicated('chunk_number', keep=False)]

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from tqdm import tqdm
import fastremap
import gpytoolbox as gyp
import trimesh
from scipy.optimize import linear_sum_assignment
from scipy import sparse
from scipy.spatial.distance import cdist
import datetime
import cloudvolume


import sys
import os
from contextlib import contextmanager
from io import StringIO

@contextmanager
def suppress_output():
    """Context manager to suppress stdout and stderr output."""
    # Save original stdout and stderr
    original_stdout = sys.stdout
    original_stderr = sys.stderr
    
    try:
        # Redirect to devnull or StringIO
        with open(os.devnull, 'w') as devnull:
            sys.stdout = devnull
            sys.stderr = devnull
            yield
    finally:
        # Restore original stdout and stderr
        sys.stdout = original_stdout
        sys.stderr = original_stderr

In [ ]:
pf_df = client.tables.proofreading_status_and_strategy(status_axon=True).query()

In [ ]:
rid = pf_df.pt_root_id.iloc[2]

In [ ]:
mesh = client.mesh.get_mesh(rid)

In [ ]:
nrn = client.get_skeleton(rid)

In [ ]:
def get_lvl2_points(
    l2ids,
    caveclient,
):
    data = caveclient.l2cache.get_l2data(l2ids, attributes=['rep_coord_nm'])
    df = pd.DataFrame(
        {
            'lvl2_id': [int(x) for x in data.keys()],
            'pt_x': [x['rep_coord_nm'][0] for x in data.values()],
            'pt_y': [x['rep_coord_nm'][1] for x in data.values()],
            'pt_z': [x['rep_coord_nm'][2] for x in data.values()],
        }
    ).set_index('lvl2_id')

    return df.loc[l2ids][['pt_x', 'pt_y', 'pt_z']].values

In [ ]:
cv = client.cave.info.segmentation_cloudvolume()
cv.image.lru.resize(10*1024)

In [ ]:
def chunk_to_nm(xyz_ch, cv):
    """Map a chunk location to Euclidean space

    Parameters
    ----------
    xyz_ch : array-like
        Nx3 array of chunk indices
    cv : cloudvolume.CloudVolume
        CloudVolume object associated with the chunked space
    voxel_resolution : list, optional
        Voxel resolution, by default [4, 4, 40]

    Returns
    -------
    np.array
        Nx3 array of spatial points
    """
    base_location = (cv.meta.voxel_offset(0) * cv.mip_resolution(0))
    x_vox = np.atleast_2d(xyz_ch) * cv.meta.graph_chunk_size * cv.mip_resolution(0)
    return base_location + x_vox


def chunk_dims(cv):
    """Gets the size of a chunk in euclidean space

    Parameters
    ----------
    cv : cloudvolume.CloudVolume
        Chunkedgraph-targeted cloudvolume object

    Returns
    -------
    np.array
        3-element box dimensions of a chunk in nanometers.
    """
    dims = chunk_to_nm([1, 1, 1], cv) - chunk_to_nm([0, 0, 0], cv)
    return np.squeeze(dims)

def adjust_for_draco(
    vals,
    draco_size,
):
    "Adjust grid locations to align with the discrete draco grid"
    return draco_size * np.floor(vals / draco_size)

def make_chunk_bbox(l2ids, cv, adjust_draco=True):
    chunk_numbers = [int(cv.meta.decode_chunk_position_number(l)) for l in l2ids]
    chunk_grid = np.array([np.array(cv.meta.decode_chunk_position(l)) for l in l2ids])
    chunk_start = chunk_to_nm(chunk_grid, cv)
    chunk_end = chunk_start + chunk_dims(cv)

    if adjust_draco:
        draco_size = cv.meta.get_draco_grid_size(0)
        chunk_start = adjust_for_draco(chunk_start, draco_size)
        chunk_end = adjust_for_draco(chunk_end, draco_size)

    df = pd.DataFrame(
        {
            'l2id': l2ids.astype(int),
            'chunk_x': chunk_grid[:, 0],
            'chunk_y': chunk_grid[:, 1],
            'chunk_z': chunk_grid[:, 2],
            'bbox_start_x': chunk_start[:, 0],
            'bbox_start_y': chunk_start[:, 1],
            'bbox_start_z': chunk_start[:, 2],
            'bbox_end_x': chunk_end[:, 0],
            'bbox_end_y': chunk_end[:, 1],
            'bbox_end_z': chunk_end[:, 2],
            'chunk_number': chunk_numbers,
        }
    )
    return df

def chunk_dataframe(l2ids, points, cv):
    """Create a dataframe of chunk bounding boxes for a neuron

    Parameters
    ----------
    l2ids : array-like
        List of level 2 IDs
    points : pd.DataFrame
        DataFrame containing point coordinates
    cv : cloudvolume.CloudVolume
        CloudVolume object associated with the chunked space

    Returns
    -------
    pd.DataFrame
        DataFrame containing bounding boxes for each chunk in the neuron
    """
    df = make_chunk_bbox(l2ids, cv)
    pt_df = pd.DataFrame(
        {
            'l2id': l2ids.astype(int),
            'pt_x': points[:, 0],
            'pt_y': points[:, 1],
            'pt_z': points[:, 2],
        }
    )
    return df.merge(
        pt_df,
        on='l2id',
        how='left',
    )


def bbox_mask(
    row,
    vertices,
    inclusive=True,
):
    """Create a mask for vertices within a bounding box defined by a row of chunk_df_solo

    Parameters
    ----------
    row : pd.Series
        A row from chunk_df_solo containing bounding box coordinates
    vertices : np.ndarray
        Array of vertex positions

    Returns
    -------
    np.ndarray
        Boolean mask indicating which vertices are within the bounding box
    """
    if inclusive:
        return (
            (vertices[:, 0] >= row['bbox_start_x'])
            & (vertices[:, 0] <= row['bbox_end_x'])
            & (vertices[:, 1] >= row['bbox_start_y'])
            & (vertices[:, 1] <= row['bbox_end_y'])
            & (vertices[:, 2] >= row['bbox_start_z'])
            & (vertices[:, 2] <= row['bbox_end_z'])
        )
    else:
        return (
            (vertices[:, 0] >= row['bbox_start_x'])
            & (vertices[:, 0] < row['bbox_end_x'])
            & (vertices[:, 1] >= row['bbox_start_y'])
            & (vertices[:, 1] < row['bbox_end_y'])
            & (vertices[:, 2] >= row['bbox_start_z'])
            & (vertices[:, 2] < row['bbox_end_z'])
        )

def component_submesh(
    within_component_mask,
    mask_in,
    mask_all,
    vertices,
    faces,
):
    """Create a submesh for the specific component of the mesh"""
    component_mask = mask_in.copy()
    component_mask[mask_in] = within_component_mask

    face_mask_inclusive = mask_all[faces]
    # I want faces that have all values in the boundary inclusive mask and any values in the component mask.

    face_in_chunk = np.all(face_mask_inclusive, axis=1)
    face_touch_component = np.any(component_mask[faces], axis=1)

    component_faces = faces[face_in_chunk & face_touch_component]
    component_vertex_indices = np.unique(np.concatenate((component_faces.ravel(), np.flatnonzero(component_mask))))
    vertex_mask = np.zeros(vertices.shape[0], dtype=bool)
    vertex_mask[component_vertex_indices] = True
    relabel = {v: k for k, v in enumerate(np.flatnonzero(vertex_mask))}
    component_faces = fastremap.remap(component_faces, relabel)
    return vertices[component_vertex_indices], component_faces, component_mask

def create_component_dict(chunk_rows, vertices, faces):
    mask_all = bbox_mask(chunk_rows.iloc[0], vertices, inclusive=True)
    mask_in = bbox_mask(chunk_rows.iloc[0], vertices, inclusive=False)

    minds = np.flatnonzero(mask_in)
    relabel = {v:k for k,v in enumerate(minds)}

    face_mask = mask_in[faces]
    faces_relabel = fastremap.remap(
        faces[np.all(face_mask, axis=1)],
        relabel,
    )

    # Make sure isolated vertices are included in the components
    face_identity = np.array([[ii, ii, ii] for ii in range(len(minds))])
    vertex_cc = gyp.connected_components(np.vstack([faces_relabel, face_identity]))

    # Now for each component, find the faces associated with its true vertices plus any faces that are only on the boundary of the chunk
    components = []
    for ii in np.unique(vertex_cc):
        comp_verts, comp_faces, comp_mask = component_submesh(vertex_cc == ii, mask_in, mask_all, vertices, faces)
        if comp_faces.shape[0] == 0:
            continue
        components.append(
            {
                'component_id': int(ii),
                'vertices': comp_verts,
                'faces': comp_faces,
                'mask': comp_mask,
            }
        )
    return components


# def make_chunk_mesh(
#     vertices,
#     faces,
#     mask,
#     chunk_component_df,
# ):
#     "Generate a mesh for the parts of a mesh within a chunk bounding box"

#     relabel = {v:k for k,v in chunk_component_df.to_dict()['mind'].items()}
#     face_mask = mask[faces]
#     faces_relabel = fastremap.remap(
#         faces[np.all(face_mask, axis=1)],
#         relabel,
#     )

#     edgeonly_faces = np.flatnonzero(np.sum(face_mask, axis=1) == 2)
#     bonus_edges = []
#     if len(edgeonly_faces) == 0:
#         return trimesh.Trimesh(
#             vertices=vertices[mask],
#             faces=faces_relabel,
#             validate=False,
#             process=False,
#         ), np.zeros((0,2), dtype=int)
#     for row, row_mask in zip(faces[edgeonly_faces], mask[faces[edgeonly_faces]]):
#         bonus_edges.append(row[row_mask])
#     bonus_edges = fastremap.remap(bonus_edges, relabel)

#     return trimesh.Trimesh(
#         vertices=vertices[mask],
#         faces=faces_relabel,
#         validate=False,
#         process=False,
#     ), bonus_edges

# def within_chunk_components(
#     chunk_mesh,
#     bonus_edges,
# ):
#     """Determine vertex connected components within a chunk mesh"""

#     mesh_edges = np.vstack([chunk_mesh.edges, bonus_edges])
#     A = sparse.coo_matrix(
#         (np.ones(mesh_edges.shape[0]), (mesh_edges[:,0], mesh_edges[:,1]) ),
#         shape=(chunk_mesh.vertices.shape[0], chunk_mesh.vertices.shape[0]),
#     )
#     _, comp = sparse.csgraph.connected_components(A + A.T)
#     return comp

def assign_points_to_components(
    chunk_rows,
    vertices,
    faces,
    cv,
    ts,
):
    """Assign representative points to components in a chunk mesh.

    Returns
    -------
    pd.DataFrame
        DataFrame containing the index of the representative point in the chunk_rows dataframe and the component ID.

    dict
        Dictionary mapping component IDs to masks for the vertices in the global mesh.
    """
    pts = np.array(chunk_rows[['pt_x', 'pt_y', 'pt_z']].values, dtype=float)
    components = create_component_dict(chunk_rows, vertices, faces)
    if len(components) == 0:
        return pd.DataFrame(
            {
                'representative_pt': [],
                'graph_comp': [],
            }
        ), {}
    wn_results = []
    for comp in components:
        wn_results.append(gyp.fast_winding_number(pts, comp['vertices'], comp['faces']))

    pt_assign, mesh_assign = linear_sum_assignment(
        np.array(wn_results) / np.max(wn_results),
        maximize=True
    )

    # If there are more components than points, don't assign components to the lower-scoring ones
    if len(pts) < len(components):
        mesh_assign = mesh_assign[:len(pt_assign)]
    if len(pts) > len(components):
        print(f"More points than components for chunk number {chunk_rows.iloc[0]['chunk_number']}")
        pt_assign = pt_assign[:len(mesh_assign)]
    result_df = pd.DataFrame(
        {
            'representative_pt': pt_assign.astype(int),
            'graph_comp': mesh_assign.astype(int),
        }
    )
    comp_mask_dict = {}

    if len(components) > len(pts):
        for comp in components:
            # If you have not already assigned a point to this component, use the slower cloudvolume lookup
            if comp['component_id'] not in comp_mask_dict:
                point_to_component = representative_point_via_lookup(
                    chunk_rows=chunk_rows,
                    comp=comp,
                    timestamp=ts,
                    cv=cv,
                )
                if point_to_component == -1:
                    continue
                result_df.loc[result_df.index[-1] + 1] = {'representative_pt': point_to_component, 'graph_comp': comp['component_id']}

    for comp in components:
        if comp['component_id'] in result_df['graph_comp'].values:
            comp_mask_dict[comp['component_id']] = comp['mask']

    return result_df, comp_mask_dict


def get_mesh_l2id_from_lookup(
    comp: dict,
    timestamp: datetime.datetime,
    cv: cloudvolume.CloudVolume,
    point_counts: list = [10, 50, 100, 200, 400, 1000],
    potential_l2ids: np.ndarray = None,
):

    comp_bbox  = np.vstack([np.min(comp['vertices'], axis=0), np.max(comp['vertices'], axis=0)])
    not_enough_points = True

    while not_enough_points:
        if len(point_counts) == 0:
            return -1
        N = point_counts.pop(0)
        random_points = np.random.uniform(
            low=comp_bbox[0],
            high=comp_bbox[1],
            size=(N, 3)
        )
        with suppress_output():
            pt_lookup = np.array(list(cv.scattered_points(
                random_points,
                mip=0,
                coord_resolution=[1,1,1],
                agglomerate=True,
                stop_layer=2,
                timestamp=timestamp,
            ).values()))

        if potential_l2ids is None:
            potential_l2ids = np.unique(pt_lookup)
        point_in_root = np.isin(pt_lookup, potential_l2ids)
        if np.sum(point_in_root) >= 1:
            not_enough_points = False
        elif point_in_root.sum() == 0:
            print("No points found in the root. Trying again with more points.")
            continue
        l2ids, counts = np.unique(pt_lookup[point_in_root], return_counts=True)
        return l2ids[np.argmax(counts)]

def representative_point_via_lookup(
        chunk_rows,
        comp,
        timestamp,
        cv,
        point_counts=[10, 50, 100, 200, 400, 1000],
    ):
    l2ids = chunk_rows['l2id'].values
    l2id = get_mesh_l2id_from_lookup(
        comp,
        cv=cv,
        point_counts=point_counts,
        potential_l2ids=l2ids,
        timestamp=timestamp,
    )
    if l2id == -1:
        return -1
    else:
        return int(np.flatnonzero(l2ids == l2id)[0])
    
def all_on_border(
    vertices,
    chunk_row,
):
    return np.all(
        vertices[:, 0] == chunk_row['bbox_start_x']
    ) | np.all(
        vertices[:, 1] == chunk_row['bbox_start_y']
    ) | np.all(
        vertices[:, 2] == chunk_row['bbox_start_z']
    )

def restrict_submesh(submesh):
    submesh.faces = submesh.faces[~np.all(submesh.faces==0, axis=1)]
    inds = np.unique(submesh.faces)
    submesh.vertices = submesh.vertices[inds]
    submesh.faces = fastremap.remap(
        submesh.faces,
        {v: k for k, v in enumerate(inds)},
    )
    return submesh

def process_multicomponent_chunk(
    chunk_rows: pd.DataFrame,
    vertices: np.ndarray,
    faces: np.ndarray,
    timestamp: datetime.datetime,
):
    """Process one chunk of a mesh defined by its vertices and faces
    
    Parameters
    ----------
    chunk_rows : pd.DataFrame
        DataFrame containing chunk bounding box information and vertex positions
    vertices : np.ndarray
        Array of vertex positions for the complete mesh
    faces : np.ndarray
        Array of face indices for the complete mesh

    Returns
    -------
    tuple
        A tuple containing two arrays, both with one entry for every vertex contained in the chunk bounding box:
        - `mind`: Indices of mesh vertices in the chunk
        - `l2id_index`: Indices of the representative points in the chunk as defined by the chunk_rows DataFrame
    """
    
    # To get the right mesh faces for association, we need to include the vertices on chunk bounds even if we don't plan to assign values to them
    assignment_df, component_mask = assign_points_to_components(
        chunk_rows,
        vertices,
        faces,
        timestamp,
    )

    return assignment_df, component_mask

In [ ]:
def lookup_boundary_point_from_proximity(
    vertices,
    pts,
):
    """Given a set of vertices that are only along the boundary of a chunk, find the spatially closest point to the vertices"""
    dists = cdist(pts, vertices)
    closest_indices = np.argmin(dists, axis=1)
    return 


In [ ]:
def make_chunk_submeshes(
    vertices,
    faces,
    mask_inside,
    mask_all,
    chunk_component_df,
):
    "Generate a mesh for the parts of a mesh within a chunk bounding box"

    relabel = {v:k for k,v in chunk_component_df.to_dict()['mind'].items()}
    face_mask = mask_all[faces]
    faces_relabel = fastremap.remap(
        faces[np.all(face_mask, axis=1)],
        relabel,
    )

    edgeonly_faces = np.flatnonzero(np.sum(face_mask, axis=1) == 2)
    bonus_edges = []
    if len(edgeonly_faces) == 0:
        return trimesh.Trimesh(
            vertices=vertices[mask_all],
            faces=faces_relabel,
            validate=False,
            process=False,
        ), np.zeros((0,2), dtype=int)
    for row, row_mask in zip(faces[edgeonly_faces], mask_all[faces[edgeonly_faces]]):
        bonus_edges.append(row[row_mask])
    bonus_edges = fastremap.remap(bonus_edges, relabel)

    return trimesh.Trimesh(
        vertices=vertices[mask_all],
        faces=faces_relabel,
        validate=False,
        process=False,
    ), bonus_edges


In [ ]:
l2ids = nrn.anno.lvl2_ids.df.lvl2_id.values
pts = get_lvl2_points(l2ids, client.cave)

chunk_df = chunk_dataframe(l2ids, pts, cv)

chunk_df_solo = chunk_df.drop_duplicates('chunk_number', keep=False)
mesh_label = np.zeros(mesh.vertices.shape[0], dtype=int)-1
for i, row in tqdm(chunk_df_solo.iterrows()):
    mask = bbox_mask(row, mesh.vertices)
    mesh_label[mask] = i

chunk_df_multi = chunk_df[chunk_df.duplicated('chunk_number', keep=False)]
grps = [group for _, group in chunk_df_multi.groupby('chunk_number')]

In [ ]:
chunk_rows = grps[887]
pts = chunk_rows[['pt_x', 'pt_y', 'pt_z']].values
vertices = mesh.vertices
faces = mesh.faces

In [ ]:
def component_submesh(
    within_component_mask,
    mask_in,
    mask_all,
    vertices,
    faces,
):
    """Create a submesh for the specific component of the mesh"""
    component_mask = mask_in.copy()
    component_mask[mask_in] = within_component_mask

    face_mask_inclusive = mask_all[faces]
    # I want faces that have all values in the boundary inclusive mask and any values in the component mask.

    face_in_chunk = np.all(face_mask_inclusive, axis=1)
    face_touch_component = np.any(component_mask[faces], axis=1)

    component_faces = faces[face_in_chunk & face_touch_component]
    component_vertex_indices = np.unique(np.concatenate((component_faces.ravel(), np.flatnonzero(component_mask))))
    vertex_mask = np.zeros(vertices.shape[0], dtype=bool)
    vertex_mask[component_vertex_indices] = True
    relabel = {v: k for k, v in enumerate(np.flatnonzero(vertex_mask))}
    component_faces = fastremap.remap(component_faces, relabel)
    return vertices[component_vertex_indices], component_faces, component_mask

def create_component_dict(chunk_rows, vertices, faces):
    mask_all = bbox_mask(chunk_rows.iloc[0], vertices, inclusive=True)
    mask_in = bbox_mask(chunk_rows.iloc[0], vertices, inclusive=False)

    minds = np.flatnonzero(mask_in)
    relabel = {v:k for k,v in enumerate(minds)}

    face_mask = mask_in[faces]
    faces_relabel = fastremap.remap(
        faces[np.all(face_mask, axis=1)],
        relabel,
    )

    # Make sure isolated vertices are included in the components
    face_identity = np.array([[ii, ii, ii] for ii in range(len(minds))])
    vertex_cc = gyp.connected_components(np.vstack([faces_relabel, face_identity]))

    # Now for each component, find the faces associated with its true vertices plus any faces that are only on the boundary of the chunk
    components = []
    assigned_vertices = np.full(mask_in.shape[0], False, dtype=bool)
    for ii in np.unique(vertex_cc):
        comp_verts, comp_faces, comp_mask = component_submesh(vertex_cc == ii, mask_in, mask_all, vertices, faces)
        assigned_vertices[comp_mask] = True
        if comp_faces.shape[0] == 0:
            continue
        components.append(
            {
                'component_id': int(ii),
                'vertices': comp_verts,
                'faces': comp_faces,
                'mask': comp_mask,
            }
        )
    unassigned_mask = np.logical_and(mask_in, np.logical_not(assigned_vertices))
    return components, unassigned_mask

In [ ]:
chunk_rows.iloc[0, 'chunk_number']

In [ ]:
import datetime
import cloudvolume

def assign_points_to_components(
    chunk_rows,
    vertices,
    faces,
    cv,
    ts,
):
    """Assign representative points to components in a chunk mesh.

    Returns
    -------
    pd.DataFrame
        DataFrame containing the index of the representative point in the chunk_rows dataframe and the component ID.

    dict
        Dictionary mapping component IDs to masks for the vertices in the global mesh.
    """
    pts = np.array(chunk_rows[['pt_x', 'pt_y', 'pt_z']].values, dtype=float)
    components = create_component_dict(chunk_rows, vertices, faces)
    if len(components) == 0:
        return pd.DataFrame(
            {
                'representative_pt': [],
                'graph_comp': [],
            }
        ), {}
    wn_results = []
    for comp in components:
        wn_results.append(gyp.fast_winding_number(pts, comp['vertices'], comp['faces']))

    pt_assign, mesh_assign = linear_sum_assignment(
        np.array(wn_results) / np.max(wn_results),
        maximize=True
    )

    # If there are more components than points, don't assign components to the lower-scoring ones
    if len(pts) < len(components):
        mesh_assign = mesh_assign[:len(pt_assign)]
    if len(pts) > len(components):
        print(f"More points than components for chunk number {chunk_rows.iloc[0, 'chunk_number']}")
        pt_assign = pt_assign[:len(mesh_assign)]
    result_df = pd.DataFrame(
        {
            'representative_pt': pt_assign.astype(int),
            'graph_comp': mesh_assign.astype(int),
        }
    )
    comp_mask_dict = {}

    if len(components) > len(pts):
        for comp in components:
            # If you have not already assigned a point to this component, use the slower cloudvolume lookup
            if comp['component_id'] not in comp_mask_dict:
                point_to_component = representative_point_via_lookup(
                    chunk_rows=chunk_rows,
                    comp=comp,
                    timestamp=ts,
                    cv=cv,
                )
                if point_to_component == -1:
                    continue
                result_df.loc[result_df.index[-1] + 1] = {'representative_pt': point_to_component, 'graph_comp': comp['component_id']}

    for comp in components:
        if comp['component_id'] in result_df['graph_comp'].values:
            comp_mask_dict[comp['component_id']] = comp['mask']

    return result_df, comp_mask_dict


def get_mesh_l2id_from_lookup(
    comp: dict,
    timestamp: datetime.datetime,
    cv: cloudvolume.CloudVolume,
    point_counts: list = [10, 50, 100, 200, 400, 1000],
    potential_l2ids: np.ndarray = None,
):

    comp_bbox  = np.vstack([np.min(comp['vertices'], axis=0), np.max(comp['vertices'], axis=0)])
    not_enough_points = True

    while not_enough_points:
        if len(point_counts) == 0:
            return -1
        N = point_counts.pop(0)
        random_points = np.random.uniform(
            low=comp_bbox[0],
            high=comp_bbox[1],
            size=(N, 3)
        )
        with suppress_output():
            pt_lookup = np.array(list(cv.scattered_points(
                random_points,
                mip=0,
                coord_resolution=[1,1,1],
                agglomerate=True,
                stop_layer=2,
                timestamp=timestamp,
            ).values()))

        if potential_l2ids is None:
            potential_l2ids = np.unique(pt_lookup)
        point_in_root = np.isin(pt_lookup, potential_l2ids)
        if np.sum(point_in_root) >= 1:
            not_enough_points = False
        elif point_in_root.sum() == 0:
            print("No points found in the root. Trying again with more points.")
            continue
        l2ids, counts = np.unique(pt_lookup[point_in_root], return_counts=True)
        return l2ids[np.argmax(counts)]

def representative_point_via_lookup(
        chunk_rows,
        comp,
        timestamp,
        cv,
        point_counts=[10, 50, 100, 200, 400, 1000],
    ):
    l2ids = chunk_rows['l2id'].values
    l2id = get_mesh_l2id_from_lookup(
        comp,
        cv=cv,
        point_counts=point_counts,
        potential_l2ids=l2ids,
        timestamp=timestamp,
    )
    if l2id == -1:
        return -1
    else:
        return int(np.flatnonzero(l2ids == l2id)[0])

In [ ]:
all_out = []
for ii, grp in tqdm(enumerate(grps)):
    try:
        all_out.append(
            process_multicomponent_chunk(
                grp,
                mesh.vertices,
                mesh.faces,
            )
        )
    except Exception as e:
        print(f"Error processing chunk {ii}: {e}")
        continue

In [ ]:
chunk_rows

In [ ]:
ts

In [ ]:
assignment_df, component_mask_dict = assign_points_to_components(
    chunk_rows,
    vertices,
    faces,
    cv=cv,
    ts=ts,
)

In [ ]:
component_mask_dict

In [ ]:
id_mapping = []
for _, row in assignment_df.iterrows():
    id_mapping.append(
        {
            'l2id_idx': int(chunk_rows.index[row['representative_pt']]),
            'vertex_mask': component_mask_dict[row['graph_comp']],
        }
    )

In [ ]:
id_mapping

In [ ]:
assignment_df

In [ ]:
chunk_rows.index[0]

In [ ]:
chunc

In [ ]:
l2ids[chunk_rows.index[0]]

In [ ]:
components = create_component_dict(chunk_rows, vertices, faces)

In [ ]:
comp = components[0]



In [ ]:
ts = client.cave.chunkedgraph.get_root_timestamps(rid, latest=True)[0]

In [ ]:
blah = pd.DataFrame(
    {
        'thing': [1, 2, 3],
        'other': [4, 5, 6],
    }
)

In [ ]:
blah.loc[4] = {'thing': 7, 'other': 8} 

In [ ]:
blah

In [ ]:
blah.append(
    {'thing': 7, 'other': 8}
)

In [ ]:
chunk_rows

In [ ]:
pts = chunk_rows[['pt_x', 'pt_y', 'pt_z']].values

In [ ]:
pts / [9,9,45]

In [ ]:
point_to_mask_via_lookup(
    chunk_rows=chunk_rows,
    comp=comp,
    timestamp=ts,
    cv=cv,
)

In [ ]:
get_mesh_l2id_from_lookup(
    comp,
)

In [ ]:
is_inside = gyp.fast_winding_number(random_points, comp['vertices'], comp['faces'])

In [ ]:
pt_lookup = np.array(list(cv.scattered_points(
    inside_points,
    mip=0,
    coord_resolution=[1,1,1],
    agglomerate=True,
    stop_layer=2,
).values()))

In [ ]:
rid

In [ ]:
cv.sca

In [ ]:
random_points

In [ ]:
components

In [ ]:
chunk_rows

In [ ]:
components

In [ ]:
chunk_rows = grps[886]
assignment_df, component_masks = assign_points_to_components(chunk_rows, vertices, faces)

In [ ]:
component_masks

In [ ]:
components

In [ ]:
map_via_vertices = {}
for ii, submesh in enumerate(submeshes):
    if submesh[0].shape[0] > 0 and submesh[1].shape[0] == 0:
        map_via_vertices[ii] = submesh[0]
# map_via_faces = {}

In [ ]:
pts

In [ ]:
np.vstack(list(map_via_vertices.values())) / [9, 9, 45]

In [ ]:
pc_dist = cdist(pts,np.vstack(list(map_via_vertices.values())))

In [ ]:
(pts / [9, 9, 45]).astype(int)

In [ ]:
chunk_rows

In [ ]:
linear_sum_assignment(pc_dist)

In [ ]:
cloud_only

In [ ]:
for submesh in submeshes:
    if submesh[1].shape[0] == 0:

In [ ]:
wn_results = []
for submesh in submeshes:
    wn_results.append(gyp.fast_winding_number(pts, submesh[0], submesh[1]))

In [ ]:
# Every mesh is a row, every point is a column
mesh_assign, pt_assign = linear_sum_assignment(
    np.array(wn_results) / np.max(wn_results),
    maximize=True
)

In [ ]:
i

In [ ]:
pt_assign

In [ ]:
pt_assign

In [ ]:
wn_results

In [ ]:
linear_sum_assignment(np.array(wn_results), maximize=True)

In [ ]:
pts

In [ ]:
pts / [9,9,45]

In [ ]:
chunk_rows

In [ ]:
wn_results

In [ ]:
chunk_rows

In [ ]:
vertices[submeshes[0][0]]

In [ ]:
pts

In [ ]:
verts_mask

In [ ]:
pts

In [ ]:
submeshes

In [ ]:
np.full(False, len(minds), dtype=bool)

In [ ]:
component_mask = mask.copy()
component_mask[mask] = vertex_cc == 0

face_mask_inclusive = mask_all[faces]
# I want faces that have all values in the boundary inclusive mask and any values in the component mask.

face_in_chunk = np.all(face_mask_inclusive, axis=1)
face_touch_component = np.any(component_mask[faces], axis=1)

component_faces = faces[face_in_chunk & face_touch_component]
unique_verts = np.unique(np.concatenate((component_faces.ravel(), np.flatnonzero(vertex_cc == 0))))
component_vertices = vertices[unique_verts]

In [ ]:
component_faces

In [ ]:
835632 in np.flatnonzero(mask_all)

In [ ]:
np.flatnonzero(component_mask)

In [ ]:
np.isin(
    faces[face_touch_component],
    np.flatnonzero(mask_all),
)

In [ ]:
np.flatnonzero(face_in_chunk)

In [ ]:
mask_all = bbox_mask(chunk_rows.iloc[0], vertices, inclusive=True)
mask = bbox_mask(chunk_rows.iloc[0], vertices, inclusive=False)

chunk_component_df = pd.DataFrame(
    {
        'mind': np.flatnonzero(mask_all),
        'include_in_label': mask[mask_all],
    }
)

chunk_mesh, bonus_edges = make_chunk_mesh(vertices, faces, mask_all, chunk_component_df)

comp = within_chunk_components(chunk_mesh, bonus_edges)
chunk_component_df['graph_comp'] = comp

pts = np.array(chunk_rows[['pt_x', 'pt_y', 'pt_z']].values, dtype=float)


In [ ]:
submeshes= []
for i in np.unique(comp):
    submesh = chunk_mesh.copy()
    submesh.update_vertices(comp==i)
    # sm = restrict_submesh(submesh)
    submeshes.append(submesh)

In [ ]:
chunk_mesh.split(only_watertight=False)

In [ ]:
submeshes= []
any_all_on_border = False
for i in np.unique(comp):
    submesh = tmesh.copy()
    submesh.update_vertices(comp==i)
    sm = restrict_submesh(submesh)
    if all_on_border(sm.vertices, chunk_rows.iloc[0]):
        any_all_on_border = True
        break
    submeshes.append(sm)


In [ ]:
all_on_border(
    sm.vertices,
    chunk_rows.iloc[0]
)

In [ ]:
comp

In [ ]:
submeshes= []
for i in np.unique(comp):
    submesh = tmesh.copy()
    submesh.update_vertices(comp==i)
    submeshes.append(submesh)

In [ ]:
pts

In [ ]:
submesh = submeshes[2]
submesh.faces = submesh.faces[~np.all(submesh.faces==0, axis=1)]
gyp.fast_winding_number(
    pts,
    submesh.vertices,
    submesh.faces,
)

In [ ]:
wn_results = []
for submesh in submeshes:
    if np.all(submesh.faces==0):
        wn_results = [0] * len(pts)
    wn_results.append(gyp.fast_winding_number(pts,submesh.vertices,submesh.faces))

pt_assign, mesh_assign = linear_sum_assignment(
    1-np.array(wn_results) / np.max(wn_results)
)


In [ ]:
submeshes[2].faces

In [ ]:
submeshes[2].vertices

In [ ]:
l2ids = nrn.anno.lvl2_ids.df.lvl2_id.values
pts = get_lvl2_points(l2ids, client.cave)
chunk_df = chunk_dataframe(l2ids, pts, cv)
chunk_df_solo = chunk_df.drop_duplicates('chunk_number', keep=False)
mesh_label = np.zeros(mesh.vertices.shape[0], dtype=int)-1
for i, row in tqdm(chunk_df_solo.iterrows()):
    mask = bbox_mask(row, mesh.vertices)
    mesh_label[mask] = i
chunk_df_multi = chunk_df[chunk_df.duplicated('chunk_number', keep=False)]

In [ ]:
gr

In [ ]:
all_out = []
for ii, grp in tqdm(enumerate(grps)):
    try:
        all_out.append(
            process_multicomponent_chunk(
                grp,
                mesh.vertices,
                mesh.faces,
            )
        )
    except Exception as e:
        print(f"Error processing chunk {ii}: {e}")
        continue

In [ ]:
chunk_rows = grps[887]
vertices = mesh.vertices
faces = mesh.faces

In [ ]:
chunk_rows

In [ ]:
process_multicomponent_chunk(
    chunk_rows,
    mesh.vertices,
    mesh.faces,
)

In [ ]:
chunk_component_df = chunk_component_df.merge(
    assignment_df,
    on='graph_comp',
    how='left',
)

chunk_component_df['l2id_index'] =chunk_component_df['representative_pt'].map(
    chunk_rows.reset_index()['index'].to_dict()
)
chunk_component_df.query('include_in_label')['mind'].values, chunk_component_df.query('include_in_label')['l2id_index'].values

I had previously used both the vertices and any extra vertices on the face of the mesh, but this could over-connect the mesh via these second vertices.
Instead of this approach, I think I should actually build my submeshes from the components of the purely within-chunk vertex graph and then attach all the extra faces (without re-doing components). This approach should also likely allow me to avoid this spatial fallback.


In [ ]:
chunk_component_df.query('')

In [ ]:
import pdb
pdb.pm()

In [ ]:
mask_all = bbox_mask(chunk_rows.iloc[0], vertices, inclusive=True)
mask = bbox_mask(chunk_rows.iloc[0], vertices, inclusive=False)

chunk_component_df = pd.DataFrame(
    {
        'mind': np.flatnonzero(mask_all),
        'include_in_label': mask[mask_all],
    }
)

tmesh, bonus_edges = make_chunk_mesh(vertices, faces, mask_all, chunk_component_df)

comp = within_chunk_components(tmesh, bonus_edges)
chunk_component_df['graph_comp'] = comp

pts = np.array(chunk_rows[['pt_x', 'pt_y', 'pt_z']].values, dtype=float)


In [ ]:
assignment_df = assign_points_to_components(
    tmesh,
    comp,
    pts,
    chunk_rows.iloc[0],
)
assignment_df

In [ ]:
wn_results

In [ ]:
    assignment_df = assign_points_to_components(
        tmesh,
        comp,
        pts,
    )


In [ ]:
submeshes

In [ ]:
restrict_submesh = submeshes[2]

In [ ]:
from scipy.spatial import cKDTree

In [ ]:
kdtree = cKDTree(pts)

In [ ]:
comp[]

In [ ]:
from scipy.spatial.distance import cdist, pdist

In [ ]:
ds = cdist(
    pts,
    tmesh.vertices
)

In [ ]:
min_per_comp = [np.min(ds[:, comp==ii], axis=1) for ii in np.unique(comp)]

In [ ]:
linear_sum_assignment(np.array(min_per_comp) / np.max(np.array(min_per_comp)))

In [ ]:
pts / [9,9,45]

In [ ]:
chunk_rows.l2id.values

In [ ]:
comp

In [ ]:
comp

In [ ]:
ind

In [ ]:
for ii in np.unique(comp):
    ds[comp==ii]



In [ ]:
pts

In [ ]:
comp[ind]

In [ ]:
sm = restrict_submesh(submeshes[1])
sm.faces

In [ ]:
submeshes= []
for i in np.unique(comp):
    submesh = tmesh.copy()
    submesh.update_vertices(comp==i)
    submeshes.append(submesh)

wn_results = []
for submesh in submeshes:
    sm = restrict_submesh(submesh)
    wn_results.append(gyp.fast_winding_number(pts, sm.vertices, sm.faces))

# pt_assign, mesh_assign = linear_sum_assignment(
#     1-np.array(wn_results) / np.max(wn_results)
# )

# wn_results = []
# for submesh in submeshes:
#     wn_results.append(gyp.fast_winding_number(pts,submesh.vertices,submesh.faces))

# pt_assign, mesh_assign = linear_sum_assignment(
#     1-np.array(wn_results) / np.max(wn_results)
# )


In [ ]:
wn_results

In [ ]:
wn_results

In [ ]:
submeshes

In [ ]:
wn_results

In [ ]:
vertices = mesh.vertices
faces = mesh.faces

In [ ]:
minds, labels = process_multicomponent_chunk(chunk_rows, vertices, faces)

In [ ]:
import pdb
pdb.pm()

In [ ]:
minds

In [ ]:
l2ids[[26,27]]

In [ ]:
mask[mask_all]

In [ ]:
mask_all[mask]

In [ ]:
chunk_component_df = pd.DataFrame(
    {
        'mind': np.flatnonzero(mask),
    }
)

In [ ]:
np.array(chunk_to_nm(
    [292, 218, 13],
    cv=cv
)) / [9,9,45] + (chunk_dims(cv) / [9,9,45])

In [ ]:
np.vstack(
    chunk_to_nm(
        chunk_df[['chunk_x', 'chunk_y', 'chunk_z']],
        cv=cv
    )
)[[26,27]] / [9,9,45]

In [ ]:
np.vstack(
    chunk_to_nm(
        chunk_df[['chunk_x', 'chunk_y', 'chunk_z']],
        cv=cv
    )
)[[26,27]] / [9,9,45] + chunk_dims(cv) / [9,9,45]

In [ ]:
grps[9]

In [ ]:
all_out = []
for ii, grp in tqdm(enumerate(grps)):
    try:
        all_out.append(
            process_multicomponent_chunk(
                grp,
                mesh.vertices,
                mesh.faces,
            )
        )
    except Exception as e:
        print(f"Error processing chunk {ii}: {e}")
        continue

In [ ]:
grps[887][]

In [ ]:
chunk_rows = grps[887]
faces = mesh.faces
vertices = mesh.vertices

In [ ]:
mask_all = bbox_mask(chunk_rows.iloc[0], vertices, inclusive=True)
mask = bbox_mask(chunk_rows.iloc[0], vertices, inclusive=False)

chunk_component_df = pd.DataFrame(
    {
        'mind': np.flatnonzero(mask_all),
        'include_in_label': mask[mask_all],
    }
)

tmesh, bonus_edges = make_chunk_mesh(vertices, faces, mask_all, chunk_component_df)

comp = within_chunk_components(tmesh, bonus_edges)
chunk_component_df['graph_comp'] = comp

pts = np.array(chunk_rows[['pt_x', 'pt_y', 'pt_z']].values, dtype=float)

In [ ]:
mask_all.sum()

In [ ]:
chunk_df['bbox_end_x']

In [ ]:
chunk_rows

In [ ]:
vertices[mask][:,0] == chunk_rows.iloc[0]['bbox_start_x'], vertices[mask][:,1] == chunk_rows.iloc[0]['bbox_start_y'], vertices[mask][:,2] == chunk_rows.iloc[0]['bbox_start_z']

In [ ]:
chunk_rows.iloc[0]['bbox_start_x']

In [ ]:
vertices[mask]

In [ ]:
np.flatnonzero(mask)

In [ ]:
vertices[np.unique(faces[np.any(faces==835632, axis=1)].ravel())]

In [ ]:
pts / [9,9,45]

In [ ]:
chunk_rows[['pt_x', 'pt_y', 'pt_z']].values / [9,9,45]

In [ ]:
assignment_df = assign_points_to_components(
    tmesh,
    comp,
    pts,
)


In [ ]:
assignment_df

In [ ]:
tmesh.split(only_watertight=False)

In [ ]:
pt.s

In [ ]:
assignment_df = assign_points_to_components(
    tmesh,
    comp,
    pts,
)

chunk_component_df = chunk_component_df.merge(
    assignment_df,
    on='graph_comp',
    how='left',
)

chunk_component_df['l2id_index'] =chunk_component_df['representative_pt'].map(
    chunk_rows.reset_index()['index'].to_dict()
)


In [ ]:
import fastremap

In [ ]:
chunk_component_df.graph_comp.unique()

In [ ]:
tmesh.vertices[37]

In [ ]:
mesh.vertices[172]

In [ ]:
mesh.faces[np.any(mesh.faces == 172, axis=1)]

In [ ]:
face_mask = mask[mesh.faces[np.any(mesh.faces == 172, axis=1)]]

In [ ]:
chunk_component_df.query('mind == 160')

In [ ]:
chunk_component_df.query('graph_comp==2')

In [ ]:
tmesh.split(only_watertight=False)

In [ ]:
relabel = {v:k for k,v in chunk_component_df.to_dict()['mind'].items()}

In [ ]:
faces_relabel = fastremap.remap(
    faces,
    relabel,
)

In [ ]:
tmesh = trimesh.Trimesh(
    vertices=mesh.vertices[mask],
    faces=faces_relabel,
)

In [ ]:
submeshes = tmesh.split(only_watertight=False)

In [ ]:
pts = np.array(grp[['pt_x', 'pt_y', 'pt_z']].values, dtype=np.float32)

In [ ]:
pts

In [ ]:
submeshes

In [ ]:
from scipy.optimize import linear_sum_assignment

In [ ]:
wn_results = []
for submesh in submeshes:
    wn_results.append(gyp.fast_winding_number(pts,submesh.vertices,submesh.faces))

pt_assign, mesh_assign = linear_sum_assignment(
    1-np.array(wn_results) / np.max(wn_results)
)

In [ ]:
linear_sum_assignment?

In [ ]:
assignment

In [ ]:
wn_results

In [ ]:
linear_sum_assignment(
    wn_results
)

In [ ]:
tmesh.edgesk

In [ ]:
mask = bbox_mask(grp.iloc[0], mesh)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
ax.plot(
    tmesh.vertices[:, 0],
    tmesh.vertices[:, 2],
    marker='o',
    linestyle='None',
    markersize=1,
    alpha=0.5,
    color='blue',
)

In [ ]:
grp

In [ ]:
rows

In [ ]:
import gpytoolbox

In [ ]:
v = gpytoolbox.normalize_points(

In [ ]:
nrn.anno.lvl2_ids.df

In [ ]:
chunk_df.drop_duplicates('chunk_number')

In [ ]:
k
